# Chatbot with message summarization
## Review
We've covered how to customize graph state schema and reducer.

We've also shown a number of ways to trim or filter messages in graph state.

## Goals
Now, let's take it one step further!

Rather than just trimming or filtering messages, we'll show how to use LLMs to produce a running summary of the conversation.

This allows us to retain a compressed representation of the full conversation, rather than just removing it with trimming or filtering.

We'll incorporate this summarization into a simple Chatbot.

And we'll equip that Chatbot with memory, supporting long-running conversations without incurring high token cost / latency.

In [20]:
import os, getpass

def _set_env_(var):
    if not os.environ.get(var):
        print("--- API KEY NOT PRESENT ---")
        os.environ[var] = getpass.getpass(f"{var}: ")
_set_env_("OPENAI_API_KEY")


In [21]:
_set_env_("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"

In [22]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="qwen-qwq-32b")

In [4]:
llm.invoke("Hello")

AIMessage(content='\n<think>\nOkay, the user said "Hello". I need to respond in a friendly way. Let me think of a simple greeting. Maybe "Hi there!" and ask how I can assist them. Keep it open-ended so they feel comfortable to ask anything. I should make sure the tone is warm and approachable. Yeah, that should work.\n\nWait, should I use an emoji? The example response has a waving hand. Maybe add that to be friendly. So, "Hi there! 👋 How can I assist you today?" That sounds good. It\'s concise and invites them to share their needs. I don\'t want to overcomplicate it. Let\'s go with that.\n</think>\n\nHi there! 👋 How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 152, 'prompt_tokens': 11, 'total_tokens': 163, 'completion_time': 0.378157579, 'prompt_time': 0.002921622, 'queue_time': 0.45432022699999997, 'total_time': 0.381079201}, 'model_name': 'qwen-qwq-32b', 'system_fingerprint': 'fp_3796682456', 'finish_reason': 'stop', 'logpro

In [23]:
from langgraph.graph import MessagesState

class State(MessagesState):
    summery: str

In [24]:
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage
import json

def chat_model(state: State) -> json:
    
    summery = state.get("summery")

    if summery:
        system_message = f"Summery of the earlier conversation: {summery}"

        messages = [SystemMessage(content=system_message) + state["messages"]]
    else:
        messages = state["messages"]

    return {"messages": llm.invoke(messages)}

We'll define a node to produce a summary.

Note, here we'll use `RemoveMessage` to filter our state after we've produced the summary.

In [25]:
def summerize_conversation(state: State): 
    summery = state.get("summery", "")

    if summery:

        summery_message=(
            f"This is summery of the conversation: {summery} \n \n"
            "Extend the summery by taking into account the new messages above:"
        )

    else:
        summery_message= "Create a summery of the conversation above:"

    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summery_message)]
    response = llm.invoke(messages)

    # Delete all but the last two messages
    # to keep the context small
    delete_message = [RemoveMessage(m.id) for m in state["messages"][:-2]]
    return {"summery": response.content, "messages": delete_message }


In [26]:
from langgraph.graph import END

def should_continue(state: State):
    messages= state["messages"]
    if len(messages) > 5:
        return "summrize conversation"
    return END

In [28]:
from IPython.display import Image, display
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START

# Define a new graph
workflow = StateGraph(State)
workflow.add_node("conversation", chat_model)
workflow.add_node(summerize_conversation)

# Set the entrypoint as conversation
workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# Compile
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

ValueError: Found edge starting at unknown node 'summarize_conversation'